In [2]:
from BlackBox.lmart.lMart_utils import LMARTGridsearch
import pandas as pd


lMart_parameter = dict(
    boosting_type=["dart", "gbdt"],
    num_leaves=[2, 5, 10, 20, 30],
    max_depth=[-1],
    n_estimators=[75, 80, 100, 150, 200],
    learning_rate=[0.05, 0.08, 0.1, 0.15, 0.2],
    reg_lambda=[0.00005, 0.0001, 0.0002, 0.0003]
)

gs = LMARTGridsearch(path_dataset="../../outputs/scores.csv", random_state=841)
best_ = gs.grid_search(lMart_parameter)

100%|██████████| 720/720 [05:17<00:00,  2.27it/s, nDCG=0.797]


In [3]:
best_

(LGBMRanker(boosting_type='dart', class_weight='balanced', force_row_wise=True,
            importance_type='gain', learning_rate=0.15, metric='ndcg',
            n_estimators=120, num_leaves=5, objective='lambdarank',
            reg_lambda=0.0002, verbose=-1),
 {'boosting_type': 'dart',
  'learning_rate': 0.15,
  'max_depth': -1,
  'n_estimators': 120,
  'num_leaves': 5,
  'reg_lambda': 0.0002},
 0.7970144406836657)

In [8]:
# nDCG on test-set
gs.eval_model(model=best_[0], df=gs.test, qIds=gs.qIds_test)

0.7927933946383483

In [18]:
# random qIds
# rand_qId = gs.test["qId"].sample().values[0]
qId= 1

In [19]:
job_curricula = gs.test[gs.test["qId"] == qId]

In [20]:
y_pred = best_[0].predict(job_curricula.iloc[:,2:13])

In [21]:
y_pred = pd.DataFrame(y_pred, index=job_curricula.index, columns=["lambdas"])

In [22]:
dt_final = pd.merge(job_curricula, y_pred, left_index=True, right_index=True)

In [23]:
dt_final.sort_values("labels",ascending=False)["labels"].head(15)

1178    4
1403    4
1021    4
876     4
928     4
1007    4
999     4
1293    4
1435    4
1452    4
1433    4
1204    4
818     4
1117    4
969     4
Name: labels, dtype: int64

In [24]:
dt_final.sort_values("lambdas",ascending=False)["labels"].head(15)

876     4
1403    4
1530    2
928     4
1404    2
1573    3
1409    2
1569    2
918     3
1589    2
1314    3
1452    4
878     2
970     3
952     2
Name: labels, dtype: int64